# Optimize GRU model to forcast SPY High prices

In [2]:
!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install torch
!pip install googlesearch-python
!pip install GoogleNews
!pip install textblob
!pip install schedule
!pip install time
!pip install telebot
!pip install telethon
!pip install transformers
!pip install vadersentiment
!pip install alpha_vantage
!pip install Prophet
!pip install tensorflow
!pip install optuna
!pip install --upgrade tensorflow

570.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


580.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


589.14s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


597.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Cloning https://github.com/huggingface/peft.git to /private/var/folders/jp/6l9fcsfs0dq5_695_52tn60r0000gn/T/pip-req-build-obgxck7r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /private/var/folders/jp/6l9fcsfs0dq5_695_52tn60r0000gn/T/pip-req-build-obgxck7r
  Resolved https://github.com/huggingface/peft.git to commit e72a96f7cfdb330236c1b6dfe49be398d83f089e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


...

705.67s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


714.34s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


723.02s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


731.86s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


740.79s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [3]:
import requests
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import schedule
import time
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from GoogleNews import GoogleNews
import yfinance as yf
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

2024-07-12 09:11:20.980244: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Get Data

In [4]:
def get_numeric_data(ticker):
    # Define the ticker symbol and time period
    period = "6mo" #must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']"
    interval = "1h"

    # Retrieve the historical market data
    data = yf.download(tickers=ticker, period=period, interval=interval)

    # Retrieve the ticker info
    ticker_info = yf.Ticker(ticker).info

    # Extract the market cap
    #market_cap = ticker_info['marketCap']

    # Calculate the moving averages
    data['MA_50'] = data['Close'].rolling(window=50).mean()
    data['MA_200'] = data['Close'].rolling(window=200).mean()

    # Calculate the RSI
    delta = data['Close'].diff(1)
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    roll_up = up.rolling(window=14).mean()
    roll_down = down.rolling(window=14).mean().abs()
    RS = roll_up / roll_down
    data['RSI'] = 100.0 - (100.0 / (1.0 + RS))

    # Calculate the MACD
    data['EMA_12'] = data['Close'].ewm(span=12, adjust=False).mean()
    data['EMA_26'] = data['Close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = data['EMA_12'] - data['EMA_26']
    data['Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()

    # Create a single dataframe called numeric_df
    numeric_df = data.select_dtypes(include=['number'])

    # Reset the index
    numeric_df.reset_index(inplace=True)

    return numeric_df

ticker = 'SPY'
numeric = get_numeric_data(ticker)
numeric.head()

[*********************100%%**********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume,MA_50,MA_200,RSI,EMA_12,EMA_26,MACD,Signal
0,2024-01-12 10:30:00-05:00,476.519989,476.910095,475.230011,476.579498,476.579498,0,NaN,NaN,NaN,476.579498,476.579498,0.000000,0.000000
1,2024-01-12 11:30:00-05:00,476.579987,476.820007,475.364990,475.920013,475.920013,5283510,NaN,NaN,NaN,476.478039,476.530648,-0.052608,-0.010522
2,2024-01-12 12:30:00-05:00,475.940002,476.760010,475.609985,476.629913,476.629913,4370521,NaN,NaN,NaN,476.501404,476.538001,-0.036596,-0.015737
3,2024-01-12 13:30:00-05:00,476.640015,477.339996,476.490112,476.910004,476.910004,5334534,NaN,NaN,NaN,476.564266,476.565556,-0.001291,-0.012847
4,2024-01-12 14:30:00-05:00,476.909912,476.929993,475.760010,476.189514,476.189514,6290298,NaN,NaN,NaN,476.506612,476.537701,-0.031090,-0.016496


# Optimize by Trial & Error

## Train GRU

In [ ]:
def train_gru_forcast(df):
    # Load the data
    df = df.copy()

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the 'Close' column
    close_df = df[['High']]

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_close = scaler.fit_transform(close_df)

    # Split the data into training and testing sets
    train_size = int(len(scaled_close) * 0.8)
    test_size = len(scaled_close) - train_size
    train_close, test_close = scaled_close[0:train_size], scaled_close[train_size:len(scaled_close)]

    # Convert the data into sequences
    def create_sequences(data, seq_len):
        X, y = [], []
        for i in range(len(data) - seq_len):
            X.append(data[i:i+seq_len])  # Append the entire row
            y.append(data[i+seq_len])  # Append the entire next row
        return np.array(X), np.array(y)

    seq_len = 1
    X_train, y_train = create_sequences(train_close, seq_len)
    X_test, y_test = create_sequences(test_close, seq_len)

    # Reshape the data for GRU
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Create the GRU model
    model = Sequential()
    model.add(GRU(units=1000, return_sequences=True, input_shape=(seq_len, 1)))
    model.add(Dropout(0.27))
    model.add(GRU(units=200, return_sequences=False))
    model.add(Dropout(0.16))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)

    # Train the model
    model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test), verbose=1,
              validation_split=0.2, callbacks=[early_stopping])
    model.save('gru_model.keras')

    # Make predictions
    predictions = model.predict(X_test)

    # Rescale the predictions
    predictions_rescaled = scaler.inverse_transform(predictions)

    # Rescale the actual values
    y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

    #print("Predictions (rescaled):", predictions_rescaled)
    #print("Actual (rescaled):", y_test_rescaled)

    # Plot the results
    plt.plot(y_test_rescaled, label='Actual')
    plt.plot(predictions_rescaled, label='Predicted')
    plt.legend()
    plt.show()

ticker = 'SPY'
df = get_numeric_data(ticker)
train_gru_forcast(df)

[*********************100%%**********************]  1 of 1 completed


X_train shape: (632, 60, 1)
y_train shape: (632, 1)
X_test shape: (113, 60, 1)
y_test shape: (113, 1)


/Users/aidanlew/miniconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## GRU Forcast

In [ ]:
import tensorflow as tf
def gru_forcast(df, future_days):
    # Load the data
    df = df.copy()

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the 'Close' column
    close_df = df[['High']]

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_close = scaler.fit_transform(close_df)

    # Prepare the input data for the model
    seq_len = 1
    last_seq = scaled_close[-seq_len:]
    future_input = last_seq.reshape((1, seq_len, 1))

    # Create the GRU model
    model = Sequential()
    model.add(GRU(units=1000, return_sequences=True, input_shape=(seq_len, 1)))
    model.add(Dropout(0.27))
    model.add(GRU(units=200, return_sequences=False))
    model.add(Dropout(0.16))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Load the trained model
    model = tf.keras.models.load_model('gru_model.keras') ###############

    # Make predictions
    predictions = model.predict(future_input)

    # Rescale the predictions
    predictions_rescaled = scaler.inverse_transform(predictions)

    # Forecast future close prices
    future_close_prices = []
    for i in range(future_days):
        future_input = np.append(future_input[:, 1:, :], predictions.reshape((1, 1, 1)), axis=1)
        predictions = model.predict(future_input)
        predictions_rescaled = scaler.inverse_transform(predictions)
        future_close_prices.append(predictions_rescaled[0, 0])

    # plot future close prices
    plt.plot( future_close_prices, label='Predicted')
    plt.legend()
    plt.show()

    return future_close_prices


ticker = 'CURLF'
df = get_numeric_data(ticker)
future_close_prices = gru_forcast(df, future_days=5)
print("Future close prices:", future_close_prices)

# Add Layers

In [ ]:
def train_gru_forcast(df):
    # Load the data
    df = df.copy()

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the 'Close' column
    close_df = df[['High']]

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_close = scaler.fit_transform(close_df)

    # Split the data into training and testing sets
    train_size = int(len(scaled_close) * 0.8)
    test_size = len(scaled_close) - train_size
    train_close, test_close = scaled_close[0:train_size], scaled_close[train_size:len(scaled_close)]

    # Convert the data into sequences
    def create_sequences(data, seq_len):
        X, y = [], []
        for i in range(len(data) - seq_len):
            X.append(data[i:i+seq_len])  # Append the entire row
            y.append(data[i+seq_len])  # Append the entire next row
        return np.array(X), np.array(y)

    seq_len = 1
    X_train, y_train = create_sequences(train_close, seq_len)
    X_test, y_test = create_sequences(test_close, seq_len)

    # Reshape the data for GRU
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Create the GRU model
    model = Sequential()
    model.add(GRU(units=1000, return_sequences=True, input_shape=(seq_len, 1)))
    model.add(Dropout(0.26))
    model.add(GRU(units=200, return_sequences=True))
    model.add(Dropout(0.26))
    model.add(GRU(units=1000, return_sequences=False))
    model.add(Dropout(0.26))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)

    # Train the model
    model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test), verbose=1,
              validation_split=0.2, callbacks=[early_stopping])
    model.save('gru_model.keras')

    # Make predictions
    predictions = model.predict(X_test)

    # Rescale the predictions
    predictions_rescaled = scaler.inverse_transform(predictions)

    # Rescale the actual values
    y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

    #print("Predictions (rescaled):", predictions_rescaled)
    #print("Actual (rescaled):", y_test_rescaled)

    # Plot the results
    plt.plot(y_test_rescaled, label='Actual')
    plt.plot(predictions_rescaled, label='Predicted')
    plt.legend()
    plt.show()

ticker = 'SPY'
df = get_numeric_data(ticker)
train_gru_forcast(df)

In [ ]:
import tensorflow as tf
def gru_forcast(df, future_days):
    # Load the data
    df = df.copy()

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the 'Close' column
    close_df = df[['High']]

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_close = scaler.fit_transform(close_df)

    # Prepare the input data for the model
    seq_len = 1
    last_seq = scaled_close[-seq_len:]
    future_input = last_seq.reshape((1, seq_len, 1))

    # Create the GRU model
    model = Sequential()
    model.add(GRU(units=1000, return_sequences=True, input_shape=(seq_len, 1)))
    model.add(Dropout(0.26))
    model.add(GRU(units=200, return_sequences=True))
    model.add(Dropout(0.26))
    model.add(GRU(units=1000, return_sequences=False))
    model.add(Dropout(0.26))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=1))


    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Load the trained model
    model = tf.keras.models.load_model('gru_model.keras') ###############

    # Make predictions
    predictions = model.predict(future_input)

    # Rescale the predictions
    predictions_rescaled = scaler.inverse_transform(predictions)

    # Forecast future close prices
    future_close_prices = []
    for i in range(future_days):
        future_input = np.append(future_input[:, 1:, :], predictions.reshape((1, 1, 1)), axis=1)
        predictions = model.predict(future_input)
        predictions_rescaled = scaler.inverse_transform(predictions)
        future_close_prices.append(predictions_rescaled[0, 0])

    # plot future close prices
    plt.plot( future_close_prices, label='Predicted')
    plt.legend()
    plt.show()

    return future_close_prices

ticker = 'SPY'
df = get_numeric_data(ticker)
future_close_prices = gru_forcast(df, future_days=30)
print("Future high prices:", future_close_prices)

# Predict to Forcast

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping

def gru_forcast(df, future_days):
    # Load the data
    df = df.copy()

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the 'Close' column
    close_df = df[['High']]

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_close = scaler.fit_transform(close_df)

    # Prepare the input data for the model
    seq_len = 1
    X_train = []
    y_train = []
    for i in range(seq_len, len(scaled_close)):
        X_train.append(scaled_close[i-seq_len:i])
        y_train.append(scaled_close[i])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    
    last_seq = scaled_close[-seq_len:]
    last_seq = last_seq.reshape((1, seq_len, 1))
    
    # Create the GRU model
    model = Sequential()
    model.add(GRU(units=1000, return_sequences=True, input_shape=(seq_len, 1)))
    model.add(Dropout(0.26))
    model.add(GRU(units=200, return_sequences=True))
    model.add(Dropout(0.26))
    model.add(GRU(units=1000, return_sequences=False))
    model.add(Dropout(0.26))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    early_stopping = EarlyStopping(monitor='loss', patience=5, min_delta=0.001)

    # Train the model
    model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=1, callbacks=[early_stopping])
    model.save('gru_model_pred-for.keras')

    # Make predictions on known data
    predictions = model.predict(X_train)

    # Rescale the predictions
    predictions_rescaled = scaler.inverse_transform(predictions)

    # Plot actual and predicted prices
    plt.plot(close_df[seq_len:], label='Actual')
    plt.plot(predictions_rescaled, label='Predicted')
    plt.legend()
    plt.show()

    #future high
    future_close_prices = []
    for i in range(future_days):
        predictions = model.predict(last_seq)
        predictions_rescaled = scaler.inverse_transform(predictions)
        future_close_prices.append(predictions_rescaled[0, 0])
        predictions = predictions.reshape((1, 1, 1))  # Reshape predictions to (1, 1, 1)
        last_seq = np.append(last_seq[:, 1:, :], predictions, axis=1)

    # Plot forecasted prices
    plt.plot(close_df[seq_len:], label='Actual')
    plt.plot(predictions_rescaled, label='Predicted')
    plt.plot(range(len(close_df[seq_len:]), len(close_df[seq_len:])+future_days), future_close_prices, label='Forecasted')
    plt.legend()
    plt.show()

    return future_close_prices

ticker = 'SPY'
df = get_numeric_data(ticker)
future_close_prices = gru_forcast(df, future_days=30)
print("Future High prices:", future_close_prices)

# Add Different Layers

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Attention
from tensorflow.keras.layers import BahdanauAttention
def train_gru_forcast(df):
    # Load the data
    df = df.copy()

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the 'Close' column
    close_df = df[['High']]

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_close = scaler.fit_transform(close_df)

    # Split the data into training and testing sets
    train_size = int(len(scaled_close) * 0.8)
    test_size = len(scaled_close) - train_size
    train_close, test_close = scaled_close[0:train_size], scaled_close[train_size:len(scaled_close)]

    # Convert the data into sequences
    def create_sequences(data, seq_len):
        X, y = [], []
        for i in range(len(data) - seq_len):
            X.append(data[i:i+seq_len])  # Append the entire row
            y.append(data[i+seq_len])  # Append the entire next row
        return np.array(X), np.array(y)

    seq_len = 1
    X_train, y_train = create_sequences(train_close, seq_len)
    X_test, y_test = create_sequences(test_close, seq_len)

    # Reshape the data for GRU
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


    # Create the GRU model
    # Define the model architecture
    model = Sequential()
    model.add(GRU(units=128, return_sequences=True, input_shape=(scaled_data.shape[1], 1)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(LSTM(units=64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(BahdanauAttention())  # Add Bahdanau attention layer
    model.add(Dense(units=32))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(units=1))  # Output layer for forecasting High prices


    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)

    # Train the model
    model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test), verbose=1,
              validation_split=0.2, callbacks=[early_stopping])
    model.save('gru_model.keras')

    # Make predictions
    predictions = model.predict(X_test)

    # Rescale the predictions
    predictions_rescaled = scaler.inverse_transform(predictions)

    # Rescale the actual values
    y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

    #print("Predictions (rescaled):", predictions_rescaled)
    #print("Actual (rescaled):", y_test_rescaled)

    # Plot the results
    plt.plot(y_test_rescaled, label='Actual')
    plt.plot(predictions_rescaled, label='Predicted')
    plt.legend()
    plt.show()

ticker = 'SPY'
df = get_numeric_data(ticker)
train_gru_forcast(df)

In [ ]:
import tensorflow as tf
def gru_forcast(df, future_days):
    # Load the data
    df = df.copy()

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the 'Close' column
    close_df = df[['High']]

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_close = scaler.fit_transform(close_df)

    # Prepare the input data for the model
    seq_len = 1
    last_seq = scaled_close[-seq_len:]
    future_input = last_seq.reshape((1, seq_len, 1))

    # Define the model architecture
    model = Sequential()
    model.add(GRU(units=128, return_sequences=True, input_shape=(scaled_data.shape[1], 1)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(LSTM(units=64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(BahdanauAttention())  # Add Bahdanau attention layer
    model.add(Dense(units=32))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(units=1))  # Output layer for forecasting High prices

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Load the trained model
    model = tf.keras.models.load_model('gru_model.keras') ###############

    # Make predictions
    predictions = model.predict(future_input)

    # Rescale the predictions
    predictions_rescaled = scaler.inverse_transform(predictions)

    # Forecast future close prices
    future_close_prices = []
    for i in range(future_days):
        future_input = np.append(future_input[:, 1:, :], predictions.reshape((1, 1, 1)), axis=1)
        predictions = model.predict(future_input)
        predictions_rescaled = scaler.inverse_transform(predictions)
        future_close_prices.append(predictions_rescaled[0, 0])

    # plot future close prices
    plt.plot( future_close_prices, label='Predicted')
    plt.legend()
    plt.show()

    return future_close_prices

ticker = 'SPY'
df = get_numeric_data(ticker)
future_close_prices = gru_forcast(df, future_days=5)
print("Future close prices:", future_close_prices)

# Add Monte Carlo

## Vanilla

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping

def gru_forcast(df, future_days, num_mc_simulations):
    # Load the data
    df = df.copy()

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the 'Close' column
    close_df = df[['High']]

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_close = scaler.fit_transform(close_df)

    # Prepare the input data for the model
    seq_len = 1
    X_train = []
    y_train = []
    for i in range(seq_len, len(scaled_close)):
        X_train.append(scaled_close[i-seq_len:i])
        y_train.append(scaled_close[i])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    
    last_seq = scaled_close[-seq_len:]
    last_seq = last_seq.reshape((1, seq_len, 1))
    
    # Create the GRU model
    model = Sequential()
    model.add(GRU(units=1000, return_sequences=True, input_shape=(seq_len, 1)))
    model.add(Dropout(0.26))
    model.add(GRU(units=200, return_sequences=True))
    model.add(Dropout(0.26))
    model.add(GRU(units=1000, return_sequences=False))
    model.add(Dropout(0.26))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    early_stopping = EarlyStopping(monitor='loss', patience=5, min_delta=0.001)

    # Train the model
    model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=0, callbacks=[early_stopping])
    model.save('gru_model_pred-for.keras')

    # Make predictions on known data
    predictions = model.predict(X_train)

    # Rescale the predictions
    predictions_rescaled = scaler.inverse_transform(predictions)

    # Plot actual and predicted prices
    #plt.plot(close_df[seq_len:], label='Actual')
    #plt.plot(predictions_rescaled, label='Predicted')
    #plt.legend()
    #plt.show()

    #future high
    future_close_prices_mc = []
    for _ in range(num_mc_simulations):
        future_close_prices = []
        last_seq_mc = scaled_close[-seq_len:]
        last_seq_mc = last_seq_mc.reshape((1, seq_len, 1))
        for i in range(future_days):
            predictions = model.predict(last_seq_mc)
            predictions_rescaled = scaler.inverse_transform(predictions)
            future_close_prices.append(predictions_rescaled[0, 0])
            predictions = predictions.reshape((1, 1, 1))  # Reshape predictions to (1, 1, 1)
            last_seq_mc = np.append(last_seq_mc[:, 1:, :], predictions, axis=1)
        future_close_prices_mc.append(future_close_prices)

    # Calculate the Monte Carlo average
    future_close_prices_mc_avg = np.mean(future_close_prices_mc, axis=0)

    # Plot forecasted prices
    plt.plot(close_df[seq_len:], label='Actual')
    plt.plot(predictions_rescaled, label='Predicted')
    plt.plot(range(len(close_df[seq_len:]), len(close_df[seq_len:])+future_days), future_close_prices_mc_avg, label='Forecasted (MC Average)')
    plt.legend()
    plt.show()

    return future_close_prices_mc_avg

ticker = 'SPY'
df = get_numeric_data(ticker)
future_close_prices_mc_avg = gru_forcast(df, future_days=14, num_mc_simulations=100)
print("Future High prices (MC Average):", future_close_prices_mc_avg)

## Different Layers

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping

def gru_forcast(df, future_days, num_mc_simulations):
    # Load the data
    df = df.copy()

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the 'Close' column
    close_df = df[['High']]

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_close = scaler.fit_transform(close_df)

    # Prepare the input data for the model
    seq_len = 1
    X_train = []
    y_train = []
    for i in range(seq_len, len(scaled_close)):
        X_train.append(scaled_close[i-seq_len:i])
        y_train.append(scaled_close[i])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    
    last_seq = scaled_close[-seq_len:]
    last_seq = last_seq.reshape((1, seq_len, 1))
    
    # Create the GRU model
    model = Sequential()
    model.add(GRU(units=1000, return_sequences=True, input_shape=(seq_len, 1)))
    model.add(Dropout(0.26))
    model.add(GRU(units=200, return_sequences=True))
    model.add(Dropout(0.26))
    model.add(GRU(units=1000, return_sequences=False))
    model.add(Dropout(0.26))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    early_stopping = EarlyStopping(monitor='loss', patience=5, min_delta=0.001)

    # Train the model
    model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=0, callbacks=[early_stopping])
    model.save('gru_model_pred-for.keras')

    # Make predictions on known data
    predictions = model.predict(X_train)

    # Rescale the predictions
    predictions_rescaled = scaler.inverse_transform(predictions)

    # Plot actual and predicted prices
    #plt.plot(close_df[seq_len:], label='Actual')
    #plt.plot(predictions_rescaled, label='Predicted')
    #plt.legend()
    #plt.show()

    #future high
    future_close_prices_mc = []
    for _ in range(num_mc_simulations):
        future_close_prices = []
        last_seq_mc = scaled_close[-seq_len:]
        last_seq_mc = last_seq_mc.reshape((1, seq_len, 1))
        for i in range(future_days):
            predictions = model.predict(last_seq_mc)
            predictions_rescaled = scaler.inverse_transform(predictions)
            future_close_prices.append(predictions_rescaled[0, 0])
            predictions = predictions.reshape((1, 1, 1))  # Reshape predictions to (1, 1, 1)
            last_seq_mc = np.append(last_seq_mc[:, 1:, :], predictions, axis=1)
        future_close_prices_mc.append(future_close_prices)

    # Calculate the Monte Carlo average
    future_close_prices_mc_avg = np.mean(future_close_prices_mc, axis=0)

    # Plot forecasted prices
    plt.plot(close_df[seq_len:], label='Actual')
    plt.plot(predictions_rescaled, label='Predicted')
    plt.plot(range(len(close_df[seq_len:]), len(close_df[seq_len:])+future_days), future_close_prices_mc_avg, label='Forecasted (MC Average)')
    plt.legend()
    plt.show()

    return future_close_prices_mc_avg

ticker = 'SPY'
df = get_numeric_data(ticker)
future_close_prices_mc_avg = gru_forcast(df, future_days=14, num_mc_simulations=100)
print("Future High prices (MC Average):", future_close_prices_mc_avg)